In [1]:


# using Pkg
# Pkg.add("BenchmarkTools")
# Pkg.add("Distributions")
# Pkg.add("CxxWrap")
# Pkg.add("JLD")
# Pkg.add("ImageIO")
# Pkg.add("CircularArrays")
# Pkg.add("OffsetArrays")
# Pkg.add("TensorOperations")
# Pkg.add("TensorCast")
# Pkg.add("Tullio")
# Pkg.add("DiffEqOperators")
# Pkg.add("BandedMatrices")
# Pkg.add("PyPlot")
# Pkg.add("Plots")
# Pkg.add("FFTW")
# Pkg.add("SciPy")
# Pkg.add("GR")
# Pkg.add("Statistics")

import GR
using BenchmarkTools
using Distributions
using Random
using Plots
using PyCall
using PyPlot
#using Images
using CircularArrays
using CxxWrap
using FFTW
using SciPy
using JLD
using Statistics
stats = pyimport("scipy.stats")
np = pyimport("numpy")
cv2 = pyimport("cv2")



PyObject <module 'cv2' from '/usr/lib/python3/dist-packages/cv2.cpython-38-x86_64-linux-gnu.so'>

In [2]:
function Laplacian_2D!(P₁,P₂,A₁,A₂,Δx)
    B₁ = CircularArrays.CircularArray(A₁);
    B₂ = CircularArrays.CircularArray(A₂);
    Threads.@threads for 😄 ∈ CartesianIndices(P₁)
        (i,j) = Tuple(😄)
        P₁[i,j] = @fastmath (-B₁[i+2,j] + 16B₁[i+1,j] 
            + 16B₁[i-1,j] -B₁[i-2,j] 
            -B₁[i,j+2] + 16B₁[i,j+1] 
            + 16B₁[i,j-1] -B₁[i,j-2] 
            - 60B₁[i,j]) /(12Δx^2);
        P₂[i,j] = @fastmath (-B₂[i+2,j] + 16B₂[i+1,j] 
            + 16B₂[i-1,j] -B₂[i-2,j] 
            -B₂[i,j+2] + 16B₂[i,j+1] 
            + 16B₂[i,j-1] -B₂[i,j-2] 
            - 60B₂[i,j]) /(12Δx^2);  
    end
    return nothing
end



function Cores_2D!(N,angle,thr)
    s = []
    count = 0
    accept = 0.5 - 0.5*thr/100
    for 😄 ∈ 1:(N-1)
        for 🥪 ∈ 1:(N-1)
            norm₁ = (angle[😄,🥪] + π)/(2π)
            norm₂ = (angle[😄+1,🥪] + π)/(2π)
            norm₃ = (angle[😄+1,🥪+1] + π)/(2π)
            norm₄ = (angle[😄,🥪+1] + π)/(2π)

            θ₁ = min(abs(norm₂ - norm₁), 1 - abs(norm₂ - norm₁))
            θ₂ = min(abs(norm₃ - norm₂), 1 - abs(norm₃ - norm₂))
            θ₃ = min(abs(norm₄ - norm₃), 1 - abs(norm₄ - norm₃))
            θₛ = θ₁ + θ₂ + θ₃

            if θₛ > accept 
                append!(s,[[😄,🥪]])
            end
        end
    end

    if length(s) > 0
        for 🇸🇦 ∈ 1:(length(s)-1)
  
            diffᵣ = s[🇸🇦 + 1][1] - s[🇸🇦][1]
            diffₛ = s[🇸🇦 + 1][2] - s[🇸🇦][2]

            if diffᵣ == 0 && diffₛ == 1
                count += 1
            end
            if diffᵣ == 1 && diffₛ == 0
                count += 1
            end
        end
    end

    num = length(s) - count

    return num
end



function setting!(moo)
    @inbounds Threads.@threads for t in eachindex(moo)
        if moo[t] > 1
            moo[t] = 1
        end
    end
    return nothing
end

function mooing!(moo,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(moo)
        (i,j) = Tuple(😄)
        @fastmath moo[i,j] = sqrt(A₁[i,j]^2 + A₂[i,j]^2)
    end
    return nothing
end


function angler!(angle,A₁,A₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(angle)
        (i,j) = Tuple(😄)
        angle[i,j] = @fastmath atan(A₂[i,j],A₁[i,j]);
    end
    return nothing
end

function meshgrid(xin,yin)
    nx=length(xin)
    ny=length(yin)
    xout=zeros(ny,nx)
    yout=zeros(ny,nx)
    for jx=1:nx
        for ix=1:ny
            xout[ix,jx]=xin[jx]
            yout[ix,jx]=yin[ix]
        end
    end
    return (x=xout, y=yout)
end

meshgrid (generic function with 1 method)

In [3]:
function PQfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,fₐ)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₁[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₁[i,j]);
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - λ * C₂[i,j]*8.4e5 * 1e12/(3fₐ) - 2/η * Ȧ₂[i,j]);
    end
    return nothing
end

function PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ)

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    PQfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,1,fₐ)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    PQAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    
    PQvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,time + Δt,1,fₐ)

    return nothing
end



PQupdate_2D! (generic function with 1 method)

In [4]:
function EQCDfupdate_2D!(F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,Ȧ₁,Ȧ₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F₁)
        (i,j) = Tuple(😄)
        F₁[i,j] = @fastmath M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j];
        F₂[i,j] = @fastmath M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j];
    end
    return nothing
end


function EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A₁)
        (i,j) = Tuple(😄)
        A₁[i,j] = @fastmath A₁[i,j] .+ Δt .* (Ȧ₁[i,j] .+ 0.5Δt .* F₁[i,j])
        A₂[i,j] = @fastmath A₂[i,j] .+ Δt .* (Ȧ₂[i,j] .+ 0.5Δt .* F₂[i,j])
    end
    return nothing
end


function EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,C₁,C₂,A₁,A₂,η,λ,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ₁)
        (i,j) = Tuple(😄)
        Ȧ₁[i,j] = @fastmath Ȧ₁[i,j] .+ 0.5Δt .* (F₁[i,j] .+ M₁[i,j] - λ * C₁[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) + ηₓ^n *η^2 * C₂[i,j]^2/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₁[i,j])
        Ȧ₂[i,j] = @fastmath Ȧ₂[i,j] .+ 0.5Δt .* (F₂[i,j] .+ M₂[i,j] - λ * C₂[i,j] * η^2 * (A₁[i,j]^2 .+ A₂[i,j]^2 .- 1) - ηₓ^n *η^2 * C₁[i,j]*C₂[i,j]/((A₁[i,j]^2 .+ A₂[i,j]^2)^1.5) - 2/η * Ȧ₂[i,j])
    end
    return nothing
end


function EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,time,fₐ,r,s)

    n = 1

    #λ = (fₐ/mass(fₐ))^2;

    λᵪ = [1024 1448 3072 3584 5504];
    #ηₓ,η = ηtime(time,fₐ);
    λ = λᵪ[r]

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end
    

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    EQCDfupdate_2D!(F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂,Ȧ₁,Ȧ₂,time,λ,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    EQCDAupdate_2D!(A₁,A₂,Δt,Ȧ₁,Ȧ₂,F₁,F₂)

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = ηtime(time+Δt,fₐ);

    EQCDvelupdate_2D!(Ȧ₁,Ȧ₂,Δt,F₁,F₂,M₁,M₂,A₁,A₂,A₁,A₂, time + Δt,λ,ηₓ,n)

    return nothing
end

EQCDupdate_2D! (generic function with 1 method)

In [5]:
function LLaplacian_2D!(P,A,Δx)
    B = CircularArray(A);
    Threads.@threads for 😄 ∈ CartesianIndices(P)
        (i,j) = Tuple(😄)
        P[i,j] = @fastmath (-B[i+2,j] + 16B[i+1,j] 
            + 16B[i-1,j] -B[i-2,j] 
            -B[i,j+2] + 16B[i,j+1] 
            + 16B[i,j-1] -B[i,j-2] 
            - 60B[i,j]) /(12Δx^2);
    end
    return nothing
end


function Lfupdate_2D!(F,M,A,Ȧ,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(F)
        (i,j) = Tuple(😄)
        F[i,j] = @fastmath M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j];
    end
    return nothing
end


function LAupdate_2D!(A,Δt,Ȧ,F)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(A)
        (i,j) = Tuple(😄)
        A[i,j] = @fastmath A[i,j] .+ Δt .* (Ȧ[i,j] .+ 0.5Δt .* F[i,j])
    end
    return nothing
end


function Lvelupdate_2D!(Ȧ,Δt,F,M,A,η,ηₓ,n)
    @inbounds Threads.@threads for 😄 ∈ CartesianIndices(Ȧ)
        (i,j) = Tuple(😄)
        Ȧ[i,j] = @fastmath Ȧ[i,j] .+ 0.5Δt .* (F[i,j] .+ M[i,j] - ηₓ^n * η^2 * sin(A[i,j]) - 2/η * Ȧ[i,j])
    end
    return nothing
end


function Lupdate_2D!(A,Ȧ,M,F,Δx,Δt,time,fₐ,s)

    #ηₓ,η = Lηtime(time,fₐ);
    n = 1

    ηᵪ = range(2.8,stop = 3.6,length = 5)
    ηₓ = ηᵪ[s]

    if time < ηₓ
        ηₓ = time
    end

    #F₁ .= M₁ .- a.^β .* λ .* A₁ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./time
    #F₂ .= M₂ .- a.^β .* λ .* A₂ .*(A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./time

    Lfupdate_2D!(F,M,A,Ȧ,time,ηₓ,n)

    #A₁ .= A₁ .+ Δt .* (Ȧ₁ .+ 0.5Δt .* F₁)
    #A₂ .= A₂ .+ Δt .* (Ȧ₂ .+ 0.5Δt .* F₂)

    LAupdate_2D!(A,Δt,Ȧ,F)

    LLaplacian_2D!(M,A,Δx)

    #Ȧ₁ .= Ȧ₁ .+ 0.5Δt .* (F₁ .+ M₁ .- a₁.^β .* λ .* A₁ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₁ ./ (time + Δt))
    #Ȧ₂ .= Ȧ₂ .+ 0.5Δt .* (F₂ .+ M₂ .- a₁.^β .* λ .* A₂ .* (A₁.^2 .+ A₂.^2 .- η.^2) .- α .* © .* Ȧ₂ ./ (time + Δt))

    #ηₓ,η = Lηtime(time+Δt,fₐ)

    Lvelupdate_2D!(Ȧ,Δt,F,M,A,time + Δt,ηₓ,n)

    return nothing
end


Lupdate_2D! (generic function with 1 method)

In [6]:
function FPQrun_2D!(n,N,t₀,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,repeats)
    #700 images on per 5000 updates
    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    moo = zeros(N,N);
    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    angle = zeros(N,N);

    # fₛ = Δx
    # freq_s = fₛ/2 * range(-1,stop = 1,length = N)
    # freq = fₛ/2 * range(0,stop = 1, length = N/2)
    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])

    x = 500

    tracker = []
    cores = []
    thr = 1
    
    saxion = []
    axion = []

    axenergy = []
    
    lo = 0
    
    for 🥪 ∈ 1:repeats
        
        tim = t₀


        while tim < 200*🥪 + t₀
            tim = round(tim,digits = 4);


            if lo % x == 0
                #125 pictures
                mooing!(moo,A₁,A₂);
                #println(moo[1,1])
                #setting!(moo);
                angler!(angle,A₁,A₂);
                
    

                strng = Cores_2D!(N,angle,thr)
                append!(tracker,tim)
                append!(cores,strng)

                # PyPlot.imsave("PQEpoch/1"*string(i)*"/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                f_image = FFTW.fft(moo)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                # Plots.savefig(plotd,"PQEpoch/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                append!(saxion,[Abin])

                f_image = FFTW.fft(angle)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                # Plots.savefig(plotc,"PQEpoch/Angle/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")


                append!(axion,[Abins])




                f_image = FFTW.fft( ( (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2 .* fₐ^2)
                f_images = (abs.(f_image)).^2
                f_images = collect(Iterators.flatten(f_images))

                Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)


                append!(axenergy,[Abint])

                
                
                PyPlot.imsave("PQTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                
            end
            
            if tim < 250
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ)
                tim = tim + Δt
            else
                PQupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt * 250/tim,tim,fₐ)
                tim = tim + Δt * 250/tim
            end
            lo += 1
        end
     


        save("PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂)




        save("PQTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores)
        save("PQTOWN/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



    end



    return nothing
end

FPQrun_2D! (generic function with 1 method)

In [7]:
function FErun_2D!(n,N,t₁,A₁,A₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeats)


    

    M₁ = zeros(N,N);
    M₂ = zeros(N,N);

    F₁ = zeros(N,N);
    F₂ = zeros(N,N);

    Laplacian_2D!(M₁,M₂,A₁,A₂,Δx)

    moo = zeros(N,N);
    angle = zeros(N,N);
    angler!(angle,A₁,A₂);

    k_freq = fftfreq(N)*N
    kx,ky = meshgrid(k_freq,k_freq)

    knrm = sqrt.( kx.^2 + ky.^2)
    knrm = collect(Iterators.flatten(knrm))

    kbins = range(0.5, N/2+1, step = 1)
    kvals = 0.5 * (kbins[2:end] + kbins[1:end-1])
    
    x = 3

    tracker = []
    cores = []
    thr = 1
    lo = 0
    strng = 1
    mng = 0

    saxion = []
    axion = []
    axenergy = []
    
    
    Ȧ = zeros(N,N)

    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

    M = zeros(N,N);

    F = zeros(N,N);

    LLaplacian_2D!(M,angle,Δx)

    
    
    for 🥪 ∈ 1:repeats
        
        tim = t₁

        while tim < 0.5*🥪 + t₁
        #while strng != 0
            
            if strng != 0

                tim = round(tim,digits = 3);
                if lo % x == 0
                    #18 pictures
                    mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    angler!(angle,A₁,A₂);
                    
                    

                    strng = Cores_2D!(N,angle,thr)
                    
          
                    
                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")


                    f_image = FFTW.fft(moo)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])


                    f_image = FFTW.fft( ( (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2) ).^2 .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])



                    
                    if r == 1 && s == 1

                        PyPlot.imsave("QCDTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                    end



                end
                if tim < 1.8
                
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt,tim,fₐ,r,s)
                    tim = tim + Δt
                    
                else
                    EQCDupdate_2D!(A₁,A₂,Ȧ₁,Ȧ₂,M₁,M₂,F₁,F₂,Δx,Δt*(1.8/tim)^3.34,tim,fₐ,r,s)
                    tim = tim + Δt*(1.8/tim)^3.34
                    
                end
                lo += 1
                
                


                
            else
                
                if mng == 0
                    Ȧ = zeros(N,N)

                    Ȧ .= (Ȧ₂ .*  A₁ .- Ȧ₁ .* A₂)./(A₁.^2 .+ A₂.^2)

                    M = zeros(N,N);

                    F = zeros(N,N);

                    LLaplacian_2D!(M,angle,Δx)
                end


                tim = round(tim,digits = 3);
                if lo % x == 0
                    #13 pictures
                    #mooing!(moo,A₁,A₂);
                    #println(moo[1,1])
                    #setting!(moo);
                    #angler!(angle,A₁,A₂);

                    strng = Cores_2D!(N,angle,thr)

        

                    append!(tracker,tim)
                    append!(cores,strng)

                    # PyPlot.imsave("EQCD/1"*string(i)*"/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png",moo,vmin=0,vmax = 1,cmap = "gray")

                    # f_image = FFTW.fft(moo)
                    # f_images = (abs.(f_image)).^2
                    # f_images = collect(Iterators.flatten(f_images))

                    # Abin,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    # Abin = π* Abin.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotd = Plots.plot(kvals,Abin,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e1,1e8))
                    # Plots.savefig(plotd,"EQCD/Fourier/"*lpad( string(trunc(Int, lo/x - 1)) ,3,"0")*".png")

                    #append!(saxion,[Abin])

                    f_image = FFTW.fft(angle)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abins,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abins = π* Abins.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    # plotc = Plots.plot(kvals,Abins,xaxis= :log,yaxis =:log,legend = false)#, ylims = (1e6,1e11))
                    # Plots.savefig(plotc,"EQCD/Angle/"*lpad( string(trunc(Int,lo/x - 1)) ,3,"0")*".png")

                    append!(axion,[Abins])


                    f_image = FFTW.fft( Ȧ.^2 .* fₐ^2)
                    f_images = (abs.(f_image)).^2
                    f_images = collect(Iterators.flatten(f_images))

                    Abint,_,_ = stats.binned_statistic(knrm,f_images,statistic = "mean",bins = kbins)
                    Abint = π* Abint.* (kbins[2:end].^2 - kbins[1:end-1].^2)

                    append!(axenergy,[Abint])






                    if r == 1 && s == 1

                        PyPlot.imsave("QCDTOWN/"*string(i)*"/"*lpad( string(trunc(Int, lo/x )) ,3,"0")*".png",angle,vmin=-π,vmax = π,cmap = "twilight")
                    end

                    


                end
                
                if tim < 1.8
                    Lupdate_2D!(angle,Ȧ,M,F,Δx,Δt,tim,fₐ,s)
                    tim = tim + Δt
                else
                    Lupdate_2D!(angle,Ȧ,M,F,Δx,Δt*(1.8/tim)^3.34,tim,fₐ,s)
                    tim = tim + Δt*(1.8/tim)^3.34
                end
                
                lo += 1
   
                mng = 1

               
            end
           

        end

        save("QCDTOWN/Saving/"*string(i)*"/Box"*string(n)*string(🥪)*".jld", "Real", A₁,"Imaginary",A₂,"Realvel",Ȧ₁,"RealIm",Ȧ₂,"Angle",angle,"AngVel",Ȧ)



        save("QCDTOWN/Saving/"*string(i)*"/Strings"*string(n)*string(🥪)*".jld","time",tracker,"number",cores)
        save("QCDTOWN/Saving/"*string(i)*"/FFT"*string(n)*string(🥪)*".jld","saxion",saxion,"axion",axion,"axenergy",axenergy)



 

        
    end


    

    return nothing
end

FErun_2D! (generic function with 1 method)

In [8]:
# using Pkg
# ENV["PYTHON"]="/usr/bin/python3.8"
# Pkg.build("PyCall")


# using CircularArrays
# using JLD
# using FFTW
# #Pkg.add("PyCall")
# #Pkg.build("PyCall")
# #Pkg.add("SciPy")
# #Pkg.add("Conda")
# using Conda
# Conda.add("scipy")
# using PyCall
# stats = pyimport("scipy.stats")

In [9]:
@time begin


    
n = 9
N = 2^n;
println(N)

rₐ = 15
fₐ = 10 .^rₐ;
r = 1
s = 1
    
    
    
LPQ = [125 250 500 1000 2000 4000 8000 16000 32000 64000]
LQCD = LPQ/600*0.4


    

for i ∈ 9:10

    t₀ = 0.0001;
    print(string(i)*" ")


    Ȧ₁ = zeros(N,N);
    Ȧ₂ = zeros(N,N);


    μ,σ = 0, 0.1;
    C₁ = rand(Normal(μ,σ),N,N);
    C₂ = rand(Normal(μ,σ),N,N);


    Δx = round(LPQ[i]/N,digits = 4)
    Δt = 0.004
        
    repeat = 3
        

    FPQrun_2D!(n, N, t₀, C₁, C₂, Ȧ₁, Ȧ₂, Δx, Δt,fₐ,i,repeat)
        
    repeat = 3
        
        
    C₁ = load("PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Real"]
    C₂ = load("PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Imaginary"]
    Ȧ₁ = load("PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["Realvel"]
    Ȧ₂ = load("PQTOWN/Saving/"*string(i)*"/Box"*string(n)*string(repeat)*".jld")["RealIm"]
    
    t₁ = 0.4
        
    Δx = round(LQCD[i]/N,digits = 6)
    Δt = 0.001
        
    repeat = 1
        
    FErun_2D!(n,N,t₁,C₁,C₂,Ȧ₁,Ȧ₂,Δx,Δt,fₐ,i,r,s,repeat)


end
    
    

    println(' ')
end

512
9 10  
19926.540823 seconds (39.67 M allocations: 3.269 GiB, 0.01% gc time, 0.00% compilation time)


In [10]:

load("PQTOWN/Saving/1/Strings53.jld")["time"]
load("QCDTOWN/Saving/1/Strings51.jld")["time"]

167-element Vector{Any}:
 0.4
 0.403
 0.406
 0.409
 0.412
 0.415
 0.418
 0.421
 0.424
 0.427
 0.43
 0.433
 0.436
 ⋮
 0.865
 0.868
 0.871
 0.874
 0.877
 0.88
 0.883
 0.886
 0.889
 0.892
 0.895
 0.898